In [ ]:
from libs.lib import Values, TrajectoryCalculator, Planet
from libs.occultation_lib import Satellite, Occultation
from libs.interplanetary_lib import PlanetsTransOrbit

import numpy as np
from matplotlib import pyplot as plt

from pymoo.util.misc import stack
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.lhs import LHS
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

In [ ]:
calc_2sats_is_enabled = True
calc_3sats_is_enabled = True

In [ ]:
# Values
myval = Values()
# Planet
earth = Planet("Earth")
mars = Planet("Mars")
# TrajectoryCalculator
earth_calculator = TrajectoryCalculator("Earth")
sun_calculator = TrajectoryCalculator("Sun")
mars_calculator = TrajectoryCalculator("Mars")
# PlanetTrans
earth_mars = PlanetsTransOrbit(earth, mars)
sat1 = Satellite(mars, receiver_is_avalable=True)
sat2 = Satellite(mars, receiver_is_avalable=False)
sat3 = Satellite(mars, receiver_is_avalable=False)

In [ ]:
#vals
t0 = 0
t_end = 1 * 24 * 60**2
dt = 500
target_is_perigee = True
radius = mars.radius
period = 2 * np.pi / (mars.mu / (radius*2)**3)**0.5

# solve lambert from earth to mars
windows , t_H = earth_mars.calc_launch_window(2024, 4, 1, 0.001, 1)
JS_launch, _, _ = myval.convert_times_to_T_TDB(2024, 4, 1, 0, 0, 0)
duration = t_H
JS0_in = JS_launch + duration
_,_,_,_,v_planet_start,v_planet_end,v_sat_start,v_sat_end = earth_mars.trajectory_by_lambert(windows[0], duration)
v_inf_vec = v_sat_end - v_planet_end

In [ ]:
xl_array_2sats = np.array([0, radius, radius, radius, 0, 0, 0, 0, 0, radius, 0, 0, 0, 0, 0])
xu_array_2sats = np.array([360, radius*2, radius*3, radius*1.5, 1, 180, 360, 360, period, radius*1.5, 1, 180, 360, 360, period])
# ----------------------------
# X : [theta, r_h, r_a, a1, e1, i1, omega1, Omega1, t_p1, a2, e2, i2, omega2, Omega2, t_p2]
class MyProblem_2sats(Problem):
    def __init__(self):
        super().__init__(n_var=15,
                         n_obj=2,
                         n_constr=2,
                         xl=xl_array_2sats,
                         xu=xu_array_2sats,
                        )
        
    def _evaluate(self, X, out, *args, **kwargs):
        n = len(X[:,0])
        f1 = np.zeros(n)
        f2 = np.zeros(n)
        f3 = np.zeros(n)
        g1 = np.zeros(n)
        g2 = np.zeros(n)
        for i in range(0, n):
            oe_observation1 = (X[i,3], X[i,4], X[i,5], X[i,6], X[i,7], X[i,8])
            oe_observation2 = (X[i,9], X[i,10], X[i,11], X[i,12], X[i,13], X[i,14])
            sat1.init_orbit_by_orbital_elems(*oe_observation1)
            sat2.init_orbit_by_orbital_elems(*oe_observation2)
            #OccultationCalculator
            occultation = Occultation(mars,[sat1,sat2])

            dv1, dv1_01, dv1_12, dv1_23 = earth_mars.trajectory_insertion(X[i,0], X[i,1], v_inf_vec, JS0_in, X[i,2], oe_observation1, target_is_perigee, plot_is_enabled=False)
            dv2, dv2_01, dv2_12, dv2_23= earth_mars.trajectory_insertion(X[i,0], X[i,1], v_inf_vec, JS0_in, X[i,2], oe_observation2, target_is_perigee, plot_is_enabled=False)
            f1[i] = dv1 + dv2

            longitude_list, latitude_list, count = occultation.simulate_position_observed(0, t0, t_end, dt)
            spatial, time = occultation.calc_evaluation(10,10,longitude_list, latitude_list)
            f2[i] = -spatial
            # f3[i] = -s/atial
            g1[i] = radius - X[i,3] * (1 - X[i,4]) #rp > radius
            g2[i] = radius - X[i,9] * (1 - X[i,10])

        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([g1, g2])

In [ ]:
xl_array_3sats = np.array([0, radius, radius, radius, 0, 0, 0, 0, 0, radius, 0, 0, 0, 0, 0,radius, 0, 0, 0, 0, 0])
xu_array_3sats = np.array([360, radius*2, radius*3, radius*1.5, 1, 180, 360, 360, period, radius*1.5, 1, 180, 360, 360, period, radius*1.5, 1, 180, 360, 360, period])
# ----------------------------
# X : [theta, r_h, r_a, a1, e1, i1, omega1, Omega1, t_p1, a2, e2, i2, omega2, Omega2, t_p2, a3, e3, i3, omega3, Omega3, t_p3]
class MyProblem_3sats(Problem):
    def __init__(self):
        super().__init__(n_var=21,
                         n_obj=2,
                         n_constr=3,
                         xl=xl_array_3sats,
                         xu=xu_array_3sats,
                        )
        
    def _evaluate(self, X, out, *args, **kwargs):
        n = len(X[:,0])
        f1 = np.zeros(n)
        f2 = np.zeros(n)
        f3 = np.zeros(n)
        g1 = np.zeros(n)
        g2 = np.zeros(n)
        g3 = np.zeros(n)
        for i in range(0, n):
            oe_observation1 = (X[i,3], X[i,4], X[i,5], X[i,6], X[i,7], X[i,8])
            oe_observation2 = (X[i,9], X[i,10], X[i,11], X[i,12], X[i,13], X[i,14])
            oe_observation3 = (X[i,15], X[i,16], X[i,17], X[i,18], X[i,19], X[i,20])
            sat1.init_orbit_by_orbital_elems(*oe_observation1)
            sat2.init_orbit_by_orbital_elems(*oe_observation2)
            sat3.init_orbit_by_orbital_elems(*oe_observation3)
            #OccultationCalculator
            occultation = Occultation(mars,[sat1,sat2,sat3])

            dv1, dv1_01, dv1_12, dv1_23 = earth_mars.trajectory_insertion(X[i,0], X[i,1], v_inf_vec, JS0_in, X[i,2], oe_observation1, target_is_perigee, plot_is_enabled=False)
            dv2, dv2_01, dv2_12, dv2_23= earth_mars.trajectory_insertion(X[i,0], X[i,1], v_inf_vec, JS0_in, X[i,2], oe_observation2, target_is_perigee, plot_is_enabled=False)
            dv3, dv3_01, dv3_12, dv3_23= earth_mars.trajectory_insertion(X[i,0], X[i,1], v_inf_vec, JS0_in, X[i,2], oe_observation3, target_is_perigee, plot_is_enabled=False)
            # f1[i] = dv1_12 + dv1_23 + dv2_12 + dv2_23 + dv3_12 + dv3_23
            f1[i] = dv1 + dv2 + dv3

            longitude_list, latitude_list, count = occultation.simulate_position_observed(0, t0, t_end, dt)
            spatial, time = occultation.calc_evaluation(10,10,longitude_list, latitude_list)
            f2[i] = -spatial
            # f3[i] = -time
            g1[i] = radius - X[i,3] * (1 - X[i,4]) #rp > radius
            g2[i] = radius - X[i,9] * (1 - X[i,10])
            g3[i] = radius - X[i,15] * (1 - X[i,16])


        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([g1, g2, g3])

# 2sats

In [ ]:
if (calc_2sats_is_enabled):
    # 問題の定義
    problem_2sats = MyProblem_2sats()

    # アルゴリズムの初期化（NSGA-IIを使用）
    algorithm_2sats = NSGA2(
        pop_size=100,
        n_offsprings=100,
        sampling=LHS(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PolynomialMutation(eta=20),
        # eliminate_duplicates=True
    )

    # 終了条件（40世代）
    termination = get_termination("n_gen", 60)

    # 最適化の実行
    res_2sats= minimize(problem_2sats,
                    algorithm_2sats,
                    termination,
                    seed=1,
                    save_history=True,
                    verbose=True)

    # 結果の可視化
    ps_2sats = problem_2sats.pareto_set(use_cache=False, flatten=False)
    pf_2sats = problem_2sats.pareto_front(use_cache=False, flatten=False)
    pop_2sats = res_2sats.pop

    # 目的関数空間
    plot_2sats = Scatter(title = "Objective Space")
    plot_2sats.add(pop_2sats.get("F"),color="black")
    plot_2sats.add(res_2sats.F, color = "red")
    if pf_2sats is not None:
        plot_2sats.add(pf_2sats, plot_type="line", color="red", alpha=0.7)
    plot_2sats.show()

In [ ]:
print(res_2sats.F)
len(res_2sats.F)
# print(res_2sats.X)

In [ ]:
if (calc_2sats_is_enabled):
    i = 0
    X_2sats = res_2sats.X
    oe_observation1 = (X_2sats[i,3], X_2sats[i,4], X_2sats[i,5], X_2sats[i,6], X_2sats[i,7], X_2sats[i,8])
    oe_observation2 = (X_2sats[i,9], X_2sats[i,10], X_2sats[i,11], X_2sats[i,12], X_2sats[i,13], X_2sats[i,14])
    sat1.init_orbit_by_orbital_elems(*oe_observation1)
    sat2.init_orbit_by_orbital_elems(*oe_observation2)
    #OccultationCalculator
    occultation = Occultation(mars,[sat1,sat2])
    dv1, dv1_01, dv1_12, dv1_23 = earth_mars.trajectory_insertion(X_2sats[i,0], X_2sats[i,1], v_inf_vec, JS0_in, X_2sats[i,2], oe_observation1, target_is_perigee)
    dv2, dv2_01, dv2_12, dv2_23= earth_mars.trajectory_insertion(X_2sats[i,0], X_2sats[i,1], v_inf_vec, JS0_in, X_2sats[i,2], oe_observation2, target_is_perigee)
    longitude_list, latitude_list, count = occultation.simulate_position_observed(0, t0, t_end, dt)
    plt.plot(longitude_list, latitude_list,'.')
    plt.show()
    fig = plt.figure()
    ax = fig.add_subplot(111,projection = '3d')   
    mars_calculator.plot_trajectory_by_oe(*oe_observation1, ax, 'r', 'sat1')
    mars_calculator.plot_trajectory_by_oe(*oe_observation2, ax, 'b', 'sat2')

# 3sats(receiver_is_enabled = False)

In [ ]:
if (calc_3sats_is_enabled):
    # 問題の定義
    problem_3sats = MyProblem_3sats()

    # アルゴリズムの初期化（NSGA-IIを使用）
    algorithm_3sats = NSGA2(
        pop_size=100,
        n_offsprings=100,
        sampling=LHS(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PolynomialMutation(eta=20),
        # eliminate_duplicates=True
    )

    # 終了条件（40世代）
    termination_3sats = get_termination("n_gen", 60)

    # 最適化の実行
    res_3sats = minimize(problem_3sats,
                    algorithm_3sats,
                    termination_3sats,
                    seed=1,
                    save_history=True,
                    verbose=True)

    # 結果の可視化
    ps_3sats = problem_3sats.pareto_set(use_cache=False, flatten=False)
    pf_3sats = problem_3sats.pareto_front(use_cache=False, flatten=False)
    pop_3sats = res_3sats.pop

    # 目的関数空間
    plot_3sats = Scatter(title = "Objective Space")
    plot_3sats.add(pop_3sats.get("F"),color="black")
    plot_3sats.add(res_3sats.F, color = "red")
    if pf_3sats is not None:
        plot_3sats.add(pf_3sats, plot_type="line", color="red", alpha=0.7)
    plot_3sats.show()

In [ ]:
print(res_3sats.F)
# print(res_3sats.X)

In [ ]:
if (calc_3sats_is_enabled):
    i = 0
    X_3sats = res_3sats.X
    oe_observation1 = (X_3sats[i,3], X_3sats[i,4], X_3sats[i,5], X_3sats[i,6], X_3sats[i,7], X_3sats[i,8])
    oe_observation2 = (X_3sats[i,9], X_3sats[i,10], X_3sats[i,11], X_3sats[i,12], X_3sats[i,13], X_3sats[i,14])
    oe_observation3 = (X_3sats[i,15], X_3sats[i,16], X_3sats[i,17], X_3sats[i,18], X_3sats[i,19], X_3sats[i,20])
    sat1.init_orbit_by_orbital_elems(*oe_observation1)
    sat2.init_orbit_by_orbital_elems(*oe_observation2)
    sat3.init_orbit_by_orbital_elems(*oe_observation3)
    #OccultationCalculator
    occultation = Occultation(mars,[sat1,sat2,sat3])
    dv1, dv1_01, dv1_12, dv1_23 = earth_mars.trajectory_insertion(X_3sats[i,0], X_3sats[i,1], v_inf_vec, JS0_in, X_3sats[i,2], oe_observation1, target_is_perigee)
    dv2, dv2_01, dv2_12, dv2_23= earth_mars.trajectory_insertion(X_3sats[i,0], X_3sats[i,1], v_inf_vec, JS0_in, X_3sats[i,2], oe_observation2, target_is_perigee)
    dv3, dv3_01, dv3_12, dv3_23= earth_mars.trajectory_insertion(X_3sats[i,0], X_3sats[i,1], v_inf_vec, JS0_in, X_3sats[i,2], oe_observation3, target_is_perigee)
    longitude_list, latitude_list, count = occultation.simulate_position_observed(0, t0, t_end, dt)
    plt.plot(longitude_list, latitude_list,'.')
    plt.show()
    fig = plt.figure()
    ax = fig.add_subplot(111,projection = '3d')   
    mars_calculator.plot_trajectory_by_oe(*oe_observation1, ax, 'r', 'sat1')
    mars_calculator.plot_trajectory_by_oe(*oe_observation2, ax, 'b', 'sat2')
    mars_calculator.plot_trajectory_by_oe(*oe_observation3, ax, 'g', 'sat3')
    plt.show()